## Excel file import

* Read in data for yogerpresso excel file
* Assign intuitive variable names for each worksheet

In [1]:
import openpyxl as xl
workbook = xl.load_workbook(filename = 'yogerpresso_data.xlsx')

In [2]:
import Exchange_Tax_calculator

In [3]:
products = workbook['products']
inventory = workbook['inventory']
orders = workbook['orders']

## Data inserting and cleaning

* Create a status column for inventory

In [4]:
for i, cell in enumerate(inventory['B'], start = 1):
    if i == 1:
        inventory[f'C{i}'] = 'Inventory Status'
    elif cell.value > 5:
        inventory[f'C{i}'] = 'Healthy Stock'
    elif cell.value > 0:
        inventory[f'C{i}'] = 'Low Stock'
    else:
        inventory[f'C{i}'] = 'Out of Stock'

* Calculate Euro Price by manipulating column 'C' using **currency_converter** function

In [5]:
products['E1'] = 'Euro Price'

ex_rate = 0.93

for row in range(2, products.max_row + 1):
    products['E' + str(row)] = Exchange_Tax_calculator.currency_converter(
        products['C' + str(row)].value, ex_rate)

* Create new columns for **sales tax** and **total** in the orders worksheet

In [6]:
orders.insert_cols(idx=5)
orders['E1'] = 'Sales Tax'
orders.insert_cols(idx=6)
orders['F1'] = 'Total'

* Create a dictionary with all of the information contained in the 'orders' worksheet

In [7]:
order_dict = {
    orders[f'A{order}'].value:[
        orders[f'B{order}'].value,
        orders[f'C{order}'].value,
        orders[f'D{order}'].value,
        orders[f'G{order}'].value,
        str(orders[f'H{order}'].value).split(', ')
    ]
    for order in range(2, orders.max_row + 1)
}

## Sales Tax Calculation

* Apply different tax rates depending on the locations using **tax_calculator** function
* Write the amount of the **sales tax** and the **total** into the excel sheet

In [8]:
from Exchange_Tax_calculator import tax_calculator

for order in order_dict.values():
    if order[3] == 'Silver Spring':
        transaction = tax_calculator(order[2], .08)
    elif order[3] == 'German Town':
        transaction = tax_calculator(order[2], .0775)
    elif order[3] == 'Oxford':
        transaction = tax_calculator(order[2], .07)
    else:
        transaction = tax_calculator(order[2], .06)
    order.insert(3, transaction[1])
    order.insert(4, transaction[2])

In [9]:
for index, order in enumerate(order_dict.values(), start=2): 
    orders[f'E{index}'] = order[3]
    orders[f'F{index}'] = order[4]

## Saving the workbook in a new file

In [10]:
workbook.save('yogerpresso_data_cleaned.xlsx')